**Michael Bocamazo**

**Question 1**: Can we transform the given data into the format(s) necessary for a quality machine learning test?  The exploration is already given in the previous notebook.

**Date**: 2016/10/12

**Methods**: Remove unnecessary data, rescale and preprocess, find relevant forms for the outputs.

**Conclusion**: The GA feature is very important and may require a transformation of the cost.  Several features, such as the Origin, Destination, Trip Purpose, and kind of Ticket will likely require an OHE of the top 5 values.  The unnecessary columns and rows were removed.  Keeping it a multiclass problem is probably best.

In [1]:
import os, sys
import csv
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
import seaborn as sns
import copy
%matplotlib inline
sns.set_style("darkgrid", {"grid.linewidth": .5, "axes.facecolor": ".9"})

In [2]:
import ML_utils as ml_ut

Read the data from a csv into a pandas dataframe.

In [3]:
df1 = pd.read_csv('swissmetro.dat', delimiter = '\t')

In [4]:
df1.describe()

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
count,10728.000000,10728.000000,10728.0,10728.000000,10728.000000,10728.000000,10728.000000,10728.000000,10728.000000,10728.000000,...,10728.000000,10728.000000,10728.000000,10728.000000,10728.000000,10728.000000,10728.000000,10728.000000,10728.000000,10728.000000
mean,2.630034,0.630034,1.0,596.500000,2.914430,0.470638,2.888423,1.493289,0.678691,2.898490,...,166.626025,514.335477,70.100671,87.466350,670.340697,20.020507,0.118568,123.795209,78.742077,2.152778
std,0.482818,0.482818,0.0,344.116678,1.147443,0.499160,2.191100,0.708293,0.603388,1.031726,...,77.353284,1088.931881,37.431633,53.550371,1441.594614,8.161895,0.323295,88.710743,55.263663,0.632293
min,2.000000,0.000000,1.0,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,31.000000,4.000000,30.000000,8.000000,6.000000,10.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,1.0,298.750000,2.000000,0.000000,1.000000,1.000000,0.000000,2.000000,...,109.000000,58.000000,30.000000,55.000000,70.000000,10.000000,0.000000,70.000000,40.000000,2.000000
50%,3.000000,1.000000,1.0,596.500000,3.000000,0.000000,3.000000,1.000000,1.000000,3.000000,...,157.000000,94.000000,60.000000,78.000000,111.000000,20.000000,0.000000,120.000000,76.000000,2.000000
75%,3.000000,1.000000,1.0,894.250000,3.250000,1.000000,3.000000,2.000000,1.000000,4.000000,...,209.000000,170.000000,120.000000,109.000000,209.000000,30.000000,0.000000,176.000000,112.000000,3.000000
max,3.000000,1.000000,1.0,1192.000000,9.000000,1.000000,10.000000,3.000000,3.000000,6.000000,...,1049.000000,5040.000000,120.000000,796.000000,6720.000000,30.000000,1.000000,1560.000000,520.000000,3.000000


## Columns to drop
We'll make a list of the column headers that we want to drop.  Some of the data is uninformative or redundant.

In [5]:
drop_cols = []

In [6]:
df1.columns

Index(['GROUP', 'SURVEY', 'SP', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO',
       'LUGGAGE', 'AGE', 'MALE', 'INCOME', 'GA', 'ORIGIN', 'DEST', 'TRAIN_AV',
       'CAR_AV', 'SM_AV', 'TRAIN_TT', 'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO',
       'SM_HE', 'SM_SEATS', 'CAR_TT', 'CAR_CO', 'CHOICE'],
      dtype='object')

We want to know the meaningful data breakdown in each feature.  The 'GROUP' feature is equivalent to the 'SURVEY' feature, which encodes survey conducted in train (0) or on car trip (1).  These show the equilavence:

In [7]:
df1['GROUP'].value_counts()

3    6759
2    3969
Name: GROUP, dtype: int64

In [8]:
df1[df1['GROUP']==2]['SURVEY'].value_counts()

0    3969
Name: SURVEY, dtype: int64

In [9]:
df1[df1['GROUP']==3]['SURVEY'].value_counts()

1    6759
Name: SURVEY, dtype: int64

In [10]:
drop_cols += ['GROUP']

The 'TRAIN_AV' and 'SM_AV' features are not informative - always 1.

In [11]:
df1['TRAIN_AV'].value_counts()

1    10728
Name: TRAIN_AV, dtype: int64

In [12]:
df1['SM_AV'].value_counts()

1    10728
Name: SM_AV, dtype: int64

In [13]:
drop_cols += ['TRAIN_AV', 'SM_AV']

However, the 'CAR_AV' is meaningful, and different from where the survey was taken.  It encodes if the car is a possible output, so could be used for segmenting a modeling into two.  The only issue is sharing learning or weights between models, or tree subsets.

In [14]:
df1['CAR_AV'].value_counts()

1    9045
0    1683
Name: CAR_AV, dtype: int64

The column 'SP' simply means stated preference survey, is fixed at 1, and so can be removed.

In [15]:
df1['SP'].value_counts()

1    10728
Name: SP, dtype: int64

In [16]:
drop_cols += ['SP']

Sex of the traveller

In [17]:
df1['MALE'].value_counts()

1    8046
0    2682
Name: MALE, dtype: int64

Class of travel, within a train

In [18]:
df1['FIRST'].value_counts()

0    5679
1    5049
Name: FIRST, dtype: int64

Note that the class of the traveller is still encoded if the survey is given based on a car trip.

In [19]:
df1[df1['SURVEY']==1]['FIRST'].value_counts()

1    3726
0    3033
Name: FIRST, dtype: int64

### Drop the unnecessary columns

In [20]:
drop_cols

['GROUP', 'TRAIN_AV', 'SM_AV', 'SP']

In [21]:
for col in drop_cols:
    df1.drop(col, axis = 1, inplace = True)

In [22]:
df1.columns

Index(['SURVEY', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'LUGGAGE', 'AGE',
       'MALE', 'INCOME', 'GA', 'ORIGIN', 'DEST', 'CAR_AV', 'TRAIN_TT',
       'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO', 'SM_HE', 'SM_SEATS', 'CAR_TT',
       'CAR_CO', 'CHOICE'],
      dtype='object')

# Description of Remaining Features
Let's give a prose description of all the features that haven't yet been referenced.  **SURVEY** is the type of trip, train or car, for which the survey was given.  **ID** is the ID of the user - this is the session ID.  There are 1192 different sessions. Every row with the same ID has the same **CHOICE** value.  **PURPOSE** (9 categorical) was the purpose of the trip for which the survey was taken. **TICKET** (11 categorical) is the kind of ticket, round-trip, one-way, half-day, seasonal, etc.  **WHO** (4 categorical) is who pays: unknown, self, employer, half-half.  **LUGGAGE** (3 cat) 0 pieces, 1 piece, 3 several pieces.  **AGE** (6 cat) is an ordinal encoding of age, with greatest val unknown.  **INCOME** (5 cat) ordinal encoding of income level. **GA** - binary for owning an annual ticket. **ORIG** and **DEST** are 26-level cats encoding the region of travel.  They are probably too large to make useful, so while informative, they are unwieldy and could be dropped.  Then we have 3 types of straight numerical features - **TT** = Travel Time, in minutes, for each type, **CO** = cost, and **HE** = Headway or period between trains.  Cost for the car is computed as a fixed cost per unit distance.  Cost for the Train is computed based on the actual fare for an individual trip, or as the cost of the whole year with a GA - this makes the data quite hard to use, because there is a very clear bimodality in the distribution of costs.  The information it encodes isn't actually useful.  We would have to count on different cost thresholds being learnt for the GA/no GA cases.  Finally, **SM_SEATS** is a binary for the kinds of seats used on the Swiss Metro - airline style, or not.

The numerical features we probably want to keep, and we can hypothesize that they will be quite useful in choice prediction.  The origin and destination features probably have to be dropped.  Or, only keep the most frequent 5 or so values.  Purpose and Ticket might be useful, but are still fairly large for a one-hot encoding given how many samples there are.  The ordinal categorical features could be useful, but often have codings for 'unknown' that are at an extreme and so would hurt any linear model.  They could be treated as NAs and then just be sampled from the known distribution.  However, it is probably necessary to A/B test these kinds of data-filling changes.

#### Multiple Levels of Data
We could make several data sets: one with all of the features except for the truly unnecessary or redundant, and another, made beforehand for simplicity without the less likely features.  

In [23]:
df1.to_csv('SM_clean.csv', index = False)

After transformations, we can make another saved set.

# The Target
On the CHOICE value.  This probably makes sense to keep as a multioutput problem.  For every row for each ID, there is a different set of Travel Time, Costs, Headway, and other parameters that determine the desirability of each method.  

It could however be transformed into a single output problem with session-level accuracy.  So each row could be converted into a set of different alternatives, each with their own travel-related parameters, but with the same customer parameters.  The target would be a binary chosen/not chosen feature.  Then, the output of the model would be a **utility**, rather than a probability, to be **normalized** to get the fractional 'shares allocation' of this user into different alternatives.  This is somewhat complicated by the fact that the different modes aren't easily comparable.  The car mode doesn't have a headway feature, and only SM has different seat configurations.  This generates about 3x the data, but a multioutput problem naturally creates models for each output.  

**Different Style**
It may be useful to train two different models: one to predict car v. not car, all car-user data goes here, then train v. SM, to which all train-only and car but train/SM choice data goes.

We have 1192 different individuals labelling their preferences, at 10728 total combinations of alternatives presented.

In [24]:
len(set(df1['ID']))

1192

In [25]:
len(df1)

10728

### Feature Sets

Here are some possible appropriate machine learning feature sets.

In [29]:
ml_feat = ['SURVEY','FIRST','LUGGAGE','AGE','MALE','INCOME','GA','CAR_AV','TRAIN_TT',
          'TRAIN_CO','TRAIN_HE','SM_TT','SM_CO','SM_HE','SM_SEATS','CAR_TT','CAR_CO']

In [30]:
ml_feat = ['SURVEY','AGE','INCOME','GA','CAR_AV','TRAIN_TT',
          'TRAIN_CO','TRAIN_HE','SM_TT','SM_CO','SM_HE','CAR_TT','CAR_CO']

In [31]:
ml_feat = ['CAR_AV','TRAIN_TT','TRAIN_CO','TRAIN_HE','SM_TT','SM_CO','SM_HE','CAR_TT','CAR_CO']

### Eliminate Unknown Choice values

There is little point in including a class with nine samples in the model, we could add it back in later if strictly necessary for comparison between models.

In [32]:
df_k = copy.deepcopy(df1)

In [33]:
df_k = df_k[df_k['CHOICE']!=0]

The eliminates nine rows.

In [34]:
len(df_k)

10719

### Valid Minimum Cost
We want to be able to create ratio features for the costs, which might improve learning by making direct comparisons.  For all rows, there is a train and SM cost, but there is only a car cost if the user has a car.  So to calculate a valid minimum for ratio features, we must account for this.

In [35]:
def calc_valid_min(df):
    x = df1.loc[:,['SM_CO','TRAIN_CO','CAR_CO']].min(axis=1)
    for i in range(len(x)):
        if x[i]==0:
            x[i] = min(df1.loc[i,['SM_CO', 'TRAIN_CO']])
    return x            

In [36]:
df_k.loc[:,'min_CO'] = calc_valid_min(df_k)

In [37]:
ratio_price_feats = ['ratio_TRAIN_CO','ratio_SM_CO','ratio_CAR_CO']

In [38]:
df_k.loc[:,'ratio_TRAIN_CO'] = df_k['TRAIN_CO']/df_k['min_CO']

df_k.loc[:,'ratio_SM_CO'] = df_k['SM_CO']/df_k['min_CO']
df_k.loc[:,'ratio_CAR_CO'] = df_k['CAR_CO']/df_k['min_CO']

### Valid Minimum Travel Time
Similarly, we want to create a set of travel time ratio features so the comparison can be done directly.

In [39]:
def calc_valid_min2(df):
    x = df1.loc[:,['SM_TT','TRAIN_TT','CAR_TT']].min(axis=1)
    for i in range(len(x)):
        if x[i]==0:
            x[i] = min(df1.loc[i,['SM_TT', 'TRAIN_TT']])
    return x            

In [41]:
df_k.loc[:,'min_TT'] = calc_valid_min2(df_k)

In [42]:
df_k.loc[:,'ratio_TRAIN_TT'] = df_k['TRAIN_TT']/df_k['min_TT']
df_k.loc[:,'ratio_SM_TT'] = df_k['SM_TT']/df_k['min_TT']
df_k.loc[:,'ratio_CAR_TT'] = df_k['CAR_TT']/df_k['min_TT']

In [43]:
ratio_feats = ratio_price_feats + ['ratio_TRAIN_TT','ratio_SM_TT','ratio_CAR_TT']

In [44]:
ml_feat = ['SURVEY','FIRST','LUGGAGE','AGE','MALE','INCOME','GA','CAR_AV','TRAIN_TT',
          'TRAIN_CO','TRAIN_HE','SM_TT','SM_CO','SM_HE','SM_SEATS','CAR_TT','CAR_CO'] + ratio_feats
          

### Save the result
We'll save these likely transformations.

In [45]:
df_k.to_csv('SM_expand.csv', index = False)